## **Read Json file**

In [1]:
import json 

data = json.load(open("Prompt_Math/math_train.json"))
data

{'__count__': 1200,
 'data': [{'id': '1',
   'question': 'Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính',
   'choices': ['A. 80,000 : 6',
    'B. 80,000 x 6',
    'C. 80,000 : (6 x 100)',
    'D. (80,000 x 6) : 100'],
   'explanation': 'Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồng . Để tìm số tiền lỗ ta có thể lấy 80 000 chia cho 100 rồi nhân với 6 (tức là 80 000 : 100 × 6) hoặc lấy 80000 nhân với 6 rồi chia cho 100 (tức là 80 000 × 6 : 100).',
   'answer': 'D. (80,000 x 6) : 100'},
  {'id': '10',
   'question': '8 dm2 24 cm2 = ……… dm2. Số thích hợp điền vào chỗ chấm là:',
   'choices': ['A. 824', 'B. 82,4', 'C. 8,24', 'D. 0,824'],
   'explanation': 'Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = 8,24 dm2.',
   'answer': 'C. 8,24'},
  {'id': '100',
   'question': '10% của 11,5m2 là:',
   'choices': ['A. 10,15dm2', 'B. 1,5m2', 'C. 15,5m2', 'D. 1,15m2'],
   'explanation': '10% của 11,5m2 là: 11,5 ${\\times}$ 10 : 100 = 1,15 (m2).',
   '

In [2]:
n_test = 20

data_test = data["data"][:n_test]
data_test

[{'id': '1',
  'question': 'Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính',
  'choices': ['A. 80,000 : 6',
   'B. 80,000 x 6',
   'C. 80,000 : (6 x 100)',
   'D. (80,000 x 6) : 100'],
  'explanation': 'Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồng . Để tìm số tiền lỗ ta có thể lấy 80 000 chia cho 100 rồi nhân với 6 (tức là 80 000 : 100 × 6) hoặc lấy 80000 nhân với 6 rồi chia cho 100 (tức là 80 000 × 6 : 100).',
  'answer': 'D. (80,000 x 6) : 100'},
 {'id': '10',
  'question': '8 dm2 24 cm2 = ……… dm2. Số thích hợp điền vào chỗ chấm là:',
  'choices': ['A. 824', 'B. 82,4', 'C. 8,24', 'D. 0,824'],
  'explanation': 'Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = 8,24 dm2.',
  'answer': 'C. 8,24'},
 {'id': '100',
  'question': '10% của 11,5m2 là:',
  'choices': ['A. 10,15dm2', 'B. 1,5m2', 'C. 15,5m2', 'D. 1,15m2'],
  'explanation': '10% của 11,5m2 là: 11,5 ${\\times}$ 10 : 100 = 1,15 (m2).',
  'answer': 'D. 1,15m2'},
 {'id': '1004',
  'ques

In [3]:
import openai
import tqdm

## **Zero Shot Prompting**

In [4]:
def zero_shot_prompts():
    """
    Generate prompts for zero-shot question answering tasks.

    This function processes a given test dataset (`data_test`) and creates prompts for zero-shot question answering tasks.
    Each prompt consists of a task description, a question, and a list of answer choices, with the goal of selecting the
    correct answer from the choices.

    Returns:
        tuple: A tuple containing two lists.
            - prompts (List[str]): A list of prompts for each question in the dataset.
            - results (List[str]): A list of correct answers corresponding to each prompt.

    Example:
        prompts, results = zero_shot_prompts()
    """
    task_description = "Select the right answer A,B,C or D in the following question: "
    prompts = []
    results = []
    for element in data_test:
        question = element["question"]
        choices = "\n".join(element["choices"])
        prompt = "\n".join([task_description,question,choices])
        results.append(element["answer"][0])
        prompts.append(prompt)
    return prompts, results
prompts, results = zero_shot_prompts()

In [6]:
print(prompts[0])

Select the right answer A,B,C or D in the following question: 
Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính
A. 80,000 : 6
B. 80,000 x 6
C. 80,000 : (6 x 100)
D. (80,000 x 6) : 100


In [69]:
def run_batch_prompt(model_name, prompts, results):
    """
    Run a batch of prompts using an OpenAI language model and evaluate their correctness.

    This function takes a pre-trained OpenAI language model and a list of prompts, each containing a question and answer
    choices. It runs the prompts through the model and evaluates the correctness of the model's predictions based on
    the provided correct results.

    Args:
        model_name (str): The name or identifier of the pre-trained language model to use.
        prompts (List[str]): A list of prompts to be processed by the language model.
        results (List[str]): A list of correct answers corresponding to each prompt.

    Returns:
        None

    Example:
        run_batch_prompt(model_name="text-davinci-003", prompts=prompts, results=results)
    """
    total_correct = 0
    for prompt, result in tqdm.tqdm(zip(prompts,results)):
        response = openai.Completion.create(
        engine=model_name,
        prompt=prompt,
        temperature=0.7,
        top_p=1.0
        )

        answer = response.choices[0].text.strip()
        try:
            pred_choice = answer[0]
            if pred_choice == result:
                total_correct+=1
        except:
            total_correct+=0
    print("Total correct (%) = ",total_correct/n_test * 100)

run_batch_prompt(model_name = "text-davinci-003", prompts = prompts, results=results)

20it [00:10,  1.89it/s]

Total correct (%) =  45.0


## **Few Shot Prompting**

**Get the example question** 

This example question are not included in test_data

In [7]:
n_few_shot_example = 5
examples=data["data"][-n_few_shot_example:]
examples

[{'id': '990',
  'question': 'Một nhóm bạn gồm Hoa, Hồng, Huệ, Hương. Cô giáo chọn 1 bạn bất kì để làm nhóm trưởng. Cô giáo ‘không thể’ chọn bạn nào?',
  'choices': ['A. Hoa', 'B. Hồng', 'C. Đào', 'D. Hương'],
  'explanation': 'Vì trong danh sách không có bạn Đào nên cô giáo không thể chọn bạn Đào.',
  'answer': 'C. Đào'},
 {'id': '991',
  'question': 'Một ô tô đi được 60 km với vận tốc 60 km/giờ, tiếp đó ô tô đi được 60 km với vận tốc 30 km/giờ. Thời gian ô tô đã đi cả hai đoạn đường là:',
  'choices': ['A. 1,5 giờ', 'B. 2 giờ', 'C. 3 giờ', 'D. 4 giờ'],
  'explanation': 'vì đoạn đường thứ nhất ô tô đi hết 1 giờ, đoạn đường thứ hai ô tô đi hết 2 giờ nên tổng số thời gian ô tô đã đi đoạn đường là 3 giờ',
  'answer': 'C. 3 giờ'},
 {'id': '993',
  'question': 'Một ô tô đi từ A đến B với vận tốc 38,5km/giờ được quãng đường 100,1km. Thời gian ô tô đã đi là:',
  'choices': ['A. 2 giờ 1 phút',
   'B. 2 giờ 6 phút',
   'C. 2 giờ 36 phút',
   'D. 2 giờ 30 phút'],
  'explanation': 

In [8]:
def few_shot_prompts():
    """
    Generate few-shot prompts for question answering tasks.

    This function generates prompts for question answering tasks with few-shot examples. It combines a task description,
    few-shot examples (questions, choices, and answers), and the main question with choices to create prompts.

    Returns:
        tuple: A tuple containing two lists.
            - prompts (List[str]): A list of prompts for each question in the dataset.
            - results (List[str]): A list of correct answers corresponding to each prompt.

    Example:
        prompts, results = few_shot_prompts()
    """
    task_description = "Select the right answer A,B,C or D in the following question: "
    prompts = []
    results = []
    # few shot example: 
    few_shot_parts = []
    for example in examples:
        question_example = example["question"]
        choices_example = "\n".join(example["choices"])
        answer_example = example["answer"][0]
        few_shot_parts.append("\n".join(["Question: "+question_example,
                              "Choices: "+choices_example,
                              "Answer: "+answer_example]))
    few_shot_string = "\n".join(few_shot_parts)
    # print("\n".join(few_shot_parts))

    # main question: 
    for element in data_test:
        question = element["question"]
        choices = "\n".join(element["choices"])
        prompt = "\n".join([task_description,
                            few_shot_string,
                            "Question: "+question,
                            "Choices+: ",choices,
                            "Answer: "])
        results.append(element["answer"][0])
        prompts.append(prompt)
    return prompts, results

prompts, results = few_shot_prompts()

In [9]:
print(prompts[0])

Select the right answer A,B,C or D in the following question: 
Question: Một nhóm bạn gồm Hoa, Hồng, Huệ, Hương. Cô giáo chọn 1 bạn bất kì để làm nhóm trưởng. Cô giáo ‘không thể’ chọn bạn nào?
Choices: A. Hoa
B. Hồng
C. Đào
D. Hương
Answer: C
Question: Một ô tô đi được 60 km với vận tốc 60 km/giờ, tiếp đó ô tô đi được 60 km với vận tốc 30 km/giờ. Thời gian ô tô đã đi cả hai đoạn đường là:
Choices: A. 1,5 giờ
B. 2 giờ
C. 3 giờ
D. 4 giờ
Answer: C
Question: Một ô tô đi từ A đến B với vận tốc 38,5km/giờ được quãng đường 100,1km. Thời gian ô tô đã đi là:
Choices: A. 2 giờ 1 phút
B. 2 giờ 6 phút
C. 2 giờ 36 phút
D. 2 giờ 30 phút
Answer: C
Question: Một ô tô đi từ Hà Nội lúc 11 giờ trưa và đến Vinh lúc 5 giờ 30 phút chiều. Dọc đường ô tô dừng ở Ninh Bình và Thanh Hóa mỗi nơi 15 phút. Hỏi không kể thời gian dừng dọc đường, ô tô đi hết quãng đường từ Hà Nội đến Vinh mất bao nhiêu thời gian ?
Choices: A. 4 giờ 30 phút
B. 6 giờ 30 phút
C. 6 giờ 15 phút
D. 6 giờ
Answer: D
Question: M

In [48]:
def run_batch_prompt(model_name, prompts, results):
    """
    Run a batch of prompts using an OpenAI language model and evaluate their correctness.

    This function takes a pre-trained OpenAI language model and a list of prompts, each containing a question and answer
    choices. It runs the prompts through the model and evaluates the correctness of the model's predictions based on
    the provided correct results.

    Args:
        model_name (str): The name or identifier of the pre-trained language model to use.
        prompts (List[str]): A list of prompts to be processed by the language model.
        results (List[str]): A list of correct answers corresponding to each prompt.

    Returns:
        None

    Example:
        run_batch_prompt(model_name="text-davinci-003", prompts=prompts, results=results)
    """
    total_correct = 0
    for prompt, result in tqdm.tqdm(zip(prompts,results)):
        response = openai.Completion.create(
        engine=model_name,
        prompt=prompt,
        temperature=0.7,
        top_p=1.0
        )

        answer = response.choices[0].text.strip()
        try:
            pred_choice = answer[0]
            if pred_choice == result:
                total_correct+=1
        except:
            total_correct+=0
    print("Total correct (%) = ",total_correct/n_test * 100)

run_batch_prompt(model_name = "text-davinci-003", prompts = prompts, results=results)

20it [00:08,  2.38it/s]

Total correct (%) =  50.0


## **COT - Chain of Thought**

In [10]:
def cot_prompts():
    """
    Generate prompts for question answering tasks with context and explanation.

    This function generates prompts for question answering tasks with context, explanation, and answer choices.
    It combines a task description, few-shot examples (questions, choices, explanations, and answers), and the main
    question with choices to create prompts.

    Returns:
        tuple: A tuple containing two lists.
            - prompts (List[str]): A list of prompts for each question in the dataset.
            - results (List[str]): A list of correct answers corresponding to each prompt.

    Example:
        prompts, results = cot_prompts()
    """
    task_description = "Select the right answer A,B,C or D in the following question: "
    prompts = []
    results = []
    # few shot example: 
    few_shot_parts = []
    for example in examples:
        question_example = example["question"]
        choices_example = "\n".join(example["choices"])
        explain_example = example["explanation"]
        answer_example = example["answer"][0]
        few_shot_parts.append("\n".join(["Question: "+question_example,
                              "Choices: "+choices_example,
                              "Explain: "+explain_example,
                              "Answer: "+answer_example]))
    few_shot_string = "\n".join(few_shot_parts)

    # main question: 
    for element in data_test:
        question = element["question"]
        choices = "\n".join(element["choices"])
        prompt = "\n".join([task_description,
                            few_shot_string,
                            "Question: "+question,
                            "Choices+: ",choices,
                            "Explain: ",
                            "Answer: "])
        results.append(element["answer"][0])
        prompts.append(prompt)
    return prompts, results

prompts, results = cot_prompts()

In [11]:
print(prompts[0])

Select the right answer A,B,C or D in the following question: 
Question: Một nhóm bạn gồm Hoa, Hồng, Huệ, Hương. Cô giáo chọn 1 bạn bất kì để làm nhóm trưởng. Cô giáo ‘không thể’ chọn bạn nào?
Choices: A. Hoa
B. Hồng
C. Đào
D. Hương
Explain: Vì trong danh sách không có bạn Đào nên cô giáo không thể chọn bạn Đào.
Answer: C
Question: Một ô tô đi được 60 km với vận tốc 60 km/giờ, tiếp đó ô tô đi được 60 km với vận tốc 30 km/giờ. Thời gian ô tô đã đi cả hai đoạn đường là:
Choices: A. 1,5 giờ
B. 2 giờ
C. 3 giờ
D. 4 giờ
Explain: vì đoạn đường thứ nhất ô tô đi hết 1 giờ, đoạn đường thứ hai ô tô đi hết 2 giờ nên tổng số thời gian ô tô đã đi đoạn đường là 3 giờ
Answer: C
Question: Một ô tô đi từ A đến B với vận tốc 38,5km/giờ được quãng đường 100,1km. Thời gian ô tô đã đi là:
Choices: A. 2 giờ 1 phút
B. 2 giờ 6 phút
C. 2 giờ 36 phút
D. 2 giờ 30 phút
Explain: Thời gian ô tô đã đi là: 100,1 : 38,5 = 2,6 (giờ)
 Đổi 2,6 giờ = 2 giờ 36 phút.
Answer: C
Question: Một ô tô đi từ Hà N

In [67]:
def run_batch_prompt(model_name, prompts, results):
    """
    Run a batch of prompts using an OpenAI language model and evaluate their correctness.

    This function takes a pre-trained OpenAI language model and a list of prompts, each containing a question and answer
    choices. It runs the prompts through the model and evaluates the correctness of the model's predictions based on
    the provided correct results.

    Args:
        model_name (str): The name or identifier of the pre-trained language model to use.
        prompts (List[str]): A list of prompts to be processed by the language model.
        results (List[str]): A list of correct answers corresponding to each prompt.

    Returns:
        None

    Example:
        run_batch_prompt(model_name="text-davinci-003", prompts=prompts, results=results)
    """
    total_correct = 0
    for prompt, result in tqdm.tqdm(zip(prompts,results)):
        response = openai.Completion.create(
        engine=model_name,
        prompt=prompt,
        temperature=0.7,
        top_p=1.0,
        max_tokens=1000,
        )

        answer = response.choices[0].text.strip()
        try:
            index = answer.find("Answer: ") 
            pred_choice = answer[0] if index == -1 else answer[index+len("Answer: ")]
            if pred_choice == result:
                total_correct+=1
        except:
            total_correct+=0
    print("Total correct (%) = ",total_correct/n_test * 100)

run_batch_prompt(model_name = "text-davinci-003", prompts = prompts, results=results)

Total correct (%) =  60.0


## **Conclusion**

Parameters: model=`text-davinci-03`, temperature=`0.7`, top_p=`1.0`, max_tokens=`1000`.
|Prompt Type|Total Correct (%)|
|---|---|
|Zero Shot Prompting|45%|
|Few Shot Prompting|50%|
|COT (Chain of thought)|60%|